imports

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

import pandas as pd

import numpy as np

functions for preprocessing fairface metadata

In [ ]:
def merge_ycs(Y,C,binary=True):

    merged = []

    for y,c in zip(Y,C):

        l = [y] if binary else list(y)

        l.extend(c)

        merged.append(l)

    return merged

def preprocess_binary_labels(Y,C):

    return merge_ycs(Y,C)

def preprocess_categorical_labels(Y,C,num_classes):

    Y = list(keras.utils.to_categorical(Y,num_classes))

    return merge_ycs(Y,C,False)


def get_category(s,c,c1):

    c.remove(c1)

    s = s.replace(c,-1)

    s = s.replace(c1,1)

    return s.replace(-1,0)

def column_clean_up(df,sort=True):

    adult = {
                  0:'Minor',
                  1:'Adult'
    }

    race = {
                  0:'Black',
                  1:'East Asian',
                  2:'Indian',
                  3:'Latino/Hispanic',
                  4:'Middle Eastern',
                  5:'Southeast Asian',
                  6:'White'
    }

    race_sex = {
                  0:'Black Female',
                  1:'Black Male',

                  2:'East Asian Female',
                  3:'East Asian Male',

                  4:'Indian Female',
                  5:'Indian Male',

                  6:'Latino/Hispanic Female',
                  7:'Latino/Hispanic Male',

                  8:'Middle Eastern Female',
                  9:'Middle Eastern Male',

                  10:'Southeast Asian Female',
                  11:'Southeast Asian Male',

                  12:'White Female',
                  13:'White Male',

    }


    sex = {
                  0:'Female',
                  1:'Male'
    }



    df['adult'] = df.loc[:, 'age']
    df['alpha-race'] = df.loc[:, 'race']
    df['race-sex'] = df.loc[:, 'race']
    df['sex'] = df.loc[:, 'gender']

    df.loc[(df['age'] == '0-2'), 'adult'] = 0
    df.loc[(df['age'] == '3-9'), 'adult'] = 0
    df.loc[(df['age'] == '10-19'), 'adult'] = 0
    df.loc[(df['age'] == '20-29'), 'adult'] = 1
    df.loc[(df['age'] == '30-39'), 'adult'] = 1
    df.loc[(df['age'] == '40-49'), 'adult'] = 1
    df.loc[(df['age'] == '50-59'), 'adult'] = 1
    df.loc[(df['age'] == '60-69'), 'adult'] = 1
    df.loc[(df['age'] == 'more than 70'), 'adult'] = 1

    df.loc[(df['gender'] == 'Female'), 'sex'] = 0
    df.loc[(df['gender'] == 'Male'), 'sex'] = 1

    df.loc[(df['race'] == 'Black'), 'race'] = 0
    df.loc[(df['race'] == 'East Asian'), 'race'] = 1
    df.loc[(df['race'] == 'Indian'), 'race'] = 2
    df.loc[(df['race'] == 'Latino-Hispanic'), 'race'] = 3
    df.loc[(df['race'] == 'Middle Eastern'), 'race'] = 4
    df.loc[(df['race'] == 'Southeast Asian'), 'race'] = 5
    df.loc[(df['race'] == 'White'), 'race'] = 6


    df.loc[(df['race'] == 0) & (df['sex'] == 0), 'race-sex'] = 0
    df.loc[(df['race'] == 0) & (df['sex'] == 1), 'race-sex'] = 1

    df.loc[(df['race'] == 1) & (df['sex'] == 0), 'race-sex'] = 2
    df.loc[(df['race'] == 1) & (df['sex'] == 1), 'race-sex'] = 3

    df.loc[(df['race'] == 2) & (df['sex'] == 0), 'race-sex'] = 4
    df.loc[(df['race'] == 2) & (df['sex'] == 1), 'race-sex'] = 5

    df.loc[(df['race'] == 3) & (df['sex'] == 0), 'race-sex'] = 6
    df.loc[(df['race'] == 3) & (df['sex'] == 1), 'race-sex'] = 7

    df.loc[(df['race'] == 4) & (df['sex'] == 0), 'race-sex'] = 8
    df.loc[(df['race'] == 4) & (df['sex'] == 1), 'race-sex'] = 9

    df.loc[(df['race'] == 5) & (df['sex'] == 0), 'race-sex'] = 10
    df.loc[(df['race'] == 5) & (df['sex'] == 1), 'race-sex'] = 11

    df.loc[(df['race'] == 6) & (df['sex'] == 0), 'race-sex'] = 12
    df.loc[(df['race'] == 6) & (df['sex'] == 1), 'race-sex'] = 13


    df['adult'] = pd.to_numeric(df['adult'])
    df['race-sex'] = pd.to_numeric(df['race-sex'])
    df['sex'] = pd.to_numeric(df['sex'])

    return df, {'adult':adult, 'race':race, 'race-sex':race_sex, 'sex':sex}

def get_labels(df,category='race',target='race',number_of_classes=7,c_names={},verbose=False):

    cats = list(df[category].unique())

    if verbose:

        print('target:\t\t\t{}\ncategory feature:\t{}\n# of category groups:\t{}\nclasses:\t\t{}'.format(target,category,len(cats),number_of_classes))

        print('groups: ', cats)

    c_data = {}

    for c in cats:

        c_data[c] = get_category(df[category].copy(),cats.copy(),c)

    Y = list(df[target].copy())

    Y = [float(i) for i in Y]

    C = []

    for k,c in c_data.items():

        c = [float(i) for i in c]

        C.append(list(c))

        if verbose:

            print('C{}:\t\t\t{}\t\t{}'.format(k,int(sum(list(c))),c_names[k]))

    C = np.array(C)

    C = C.T

    C = C.tolist()

    return preprocess_categorical_labels(Y,C,number_of_classes) if number_of_classes > 2 else preprocess_binary_labels(Y,C)